In [1]:
from submission import Grammar, read_rules
import math
import time

In [20]:
def parse_t(self, line):
        """Parse a sentence with the current grammar.

        The grammar object must be in the Chomsky normal form.

        Parameters
        ----------
        line : str
            Space-delimited tokens of a sentence.

        Returns
        -------
        log_probs : {(start, end): {lhs: log_prob}}
            Log probability for each span with both start and end
            being inclusive, for the lhs of each matched rules.
        backpointer : {(start, end): {lhs: (partition, [rule])}}
            Backpointer for each span with both start and end
            being inclusive, for the lhs of each matched rules.
            partition represents the length of the first rhs symbol.
            The list of rules are the rules with which to produce the
            lhs.
        length : number of tokens in the input sentence.
        """
        # BEGIN_YOUR_CODE
        
        def print_tree(tree,curr,space):
            if(curr > len(tree)-1):
                return ""
            elif(len(tree[curr][1])>1):
                k = curr
                count = 0
                #print(tree[curr])
                for i in range(0,len(tree)):
                    if(len(tree[i][1])>1 and (tree[i][1][1] == tree[curr][1][1])):
                        count += 1
                        if(len(tree[i][1])>1 and (tree[i][1][1] == tree[curr][1][1] and tree[i][1][0] == tree[curr][1][1])):
                            count += 1
                    if(i>0 and tree[i][0]==tree[curr][1][1]):
                        k=i
                        count -= 1
                        if(len(tree[i][1])>1 and tree[i][1][1] == tree[curr][1][1]):
                            count -= 1
                        if(count <= 0):
                            break
                #print("k = ",k,count,tree[k])
                lhs = tree[curr+1:k]
                rhs = tree[k:len(tree)]
                spaces = " " * space
                if("BIN_" in tree[curr][0]):
                    str = '{}'.format(print_tree(lhs,0,space))
                    str += '\n{}'.format(spaces)
                    str += '{}'.format(print_tree(rhs,0,space))
                    return str
                else:
                    str = '({} '.format(tree[curr][0])
                    space += len(tree[curr][0]) + 2
                    str += '{}'.format(print_tree(lhs,0,space))
                    spaces = " " * space
                    str += '\n{}'.format(spaces)
                    str += '{})'.format(print_tree(rhs,0,space))
                    return str
            else:
                space += len(tree[curr][0]) + 2
                if(curr < len(tree) - 1 and tree[curr][1][0] == tree[curr+1][0]):
                    lhs = tree[curr+1:len(tree)]
                    if("NT_" in tree[curr][0]):
                        str = '{}'.format(print_tree(lhs,0,space))
                        return str
                    else:
                        str = '({} '.format(tree[curr][0])
                        str += '{})'.format(print_tree(lhs,0,space))
                        return str
                else:
                    if("NT_" in tree[curr][0]):
                        str = '{}'.format(tree[curr][1][0])
                        return str
                    else:
                        str = '({} {})'.format(tree[curr][0],tree[curr][1][0])
                        return str
                    
                    
        print(line)
        input = line.split()
        n = len(input)
        bestScore = {}
        backPointer = {}
        binary_rules = []
        unary_rules = []
        for rule in self.rules:
            A, rhs, log_prob = rule
            if(len(rhs)==1):
                unary_rules.append(rule)
            else:
                binary_rules.append(rule)
        
        # Initialize bestScore and backPointer
        for i in range(0,n):
            for j in range(i+1,n+1):
                temp = {}
                temp2 = {}
                for rule in self.rules:
                    A, rhs, log_prob = rule
                    temp[A] = 0.0
                    rule_list = []
                    temp2[A] = (i,rule_list)
                bestScore[(i,j)] = temp 
                backPointer[(i,j)] = temp2
        
        # Fill terminal rules
        for i in range(1,n+1):
            for rule in unary_rules:
                A, rhs, log_prob = rule
                q = math.exp(log_prob)
                if(input[i-1] in rhs and q>bestScore[(i-1,i)][A]):
                    bestScore[(i-1,i)][A] = q
                    best_rules = [rule]
                    if(len(backPointer[(i-1,i)][A][1])>0):
                        best_rules += backPointer[(i-1,i)][A][1]
                    backPointer[(i-1,i)][A] = (i-1,best_rules)
                    
            found = True
            unseen = []
            unseen += unary_rules
            LHS = []
            for X in bestScore[(i-1,i)]:
                if(bestScore[(i-1,i)][X]>0.0):
                    LHS.append(X)
            while(found):
                found = False
                for A in LHS:
                    for rule in unseen:
                        B, rhs, log_prob = rule
                        q = math.exp(log_prob)
                        if(A in rhs):
                            score = q * bestScore[(i-1,i)][A]
                            if(score > bestScore[(i-1,i)][B]):
                                bestScore[(i-1,i)][B] = score
                                best_rules = [rule]
                                if(len(backPointer[(i-1,i)][A][1])>0):
                                    best_rules += backPointer[(i-1,i)][A][1]
                                if(len(backPointer[(i-1,i)][B][1])>0):
                                    best_rules += backPointer[(i-1,i)][B][1]
                                backPointer[(i-1,i)][B] = (i-1,best_rules)
                                found = True
                                unseen.remove(rule)
                                LHS.append(B)
                                break
                    if(found):
                        break
                                
# ---------------------------------------------------------------------------------------------------------------

        # cky algorithm
        for l in range(2,n+1):
            for i in range(0,n-l+1):
                j = i + l
                for k in range(i+1,j-1+1):
                    for rule in binary_rules:
                        A, rhs, log_prob = rule
                        q = math.exp(log_prob)
                        B = rhs[0]
                        C = rhs[1]
                        if(bestScore[(i,k)][B] > 0.0 and bestScore[(k,j)][C] > 0.0):
                            score = q * bestScore[(i,k)][B] * bestScore[(k,j)][C]
                            if(score > bestScore[(i,j)][A]):
                                bestScore[(i,j)][A] = score
                                backPointer[(i,j)][A] = (k,[rule] + backPointer[(i,k)][B][1] + backPointer[(k,j)][C][1])
                
                # Add unary rules to non-terminals
                found = True
                unseen = []
                unseen += unary_rules
                LHS = []
                for X in bestScore[(i,j)]:
                    if(bestScore[(i,j)][X]>0.0):
                        LHS.append(X)
                while(found):
                    found = False
                    for A in LHS:
                        for rule in unseen:
                            B, rhs, log_prob = rule
                            if(A in rhs):
                                q = math.exp(log_prob)
                                score = q * bestScore[(i,j)][A]
                                if(score > bestScore[(i,j)][B]):
                                    bestScore[(i,j)][B] = score
                                    backPointer[(i,j)][B] = (i,[rule] + backPointer[(i,j)][A][1])
                                    found = True
                                    unseen.remove(rule)
                                    LHS.append(B)
                                    break
                        if(found):
                            break
        
        if(n==0):
            found = False
        elif(bestScore[(0,n)]['ROOT'] == 0.0):
            print("NONE")
        else:
            tree = backPointer[(0,n)]['ROOT'][1]
            for t in tree:
                print(t)
            print(print_tree(tree,0,0))
            print(math.log(bestScore[(0,n)]['ROOT']))
            
            

In [21]:
def parse(grammar_filename, filename):
    rules = read_rules(grammar_filename)
    grammar = Grammar(rules)
    cnf_grammar = grammar.get_cnf()
    for rule in cnf_grammar.rules:
        lhs, rhs, log_prob = rule
        if("S" == lhs and "NP" in rhs):
            print(rule)
    with open(filename, 'r') as f:
        lines = f.readlines()

    #for line in lines:
    parse_t(cnf_grammar, lines[2])

def test_arith():
    parse('data/arith.gr', 'data/arith.sen')

def test_wallstreet():
    parse('data/wallstreet.gr', 'data/wallstreet.sen')

start_time = time.time()   
#test_arith()
test_wallstreet()
print("--- %s seconds ---" % (time.time() - start_time))

S -> NP BIN_"S->NP,VP,PUNC:,PUNC."_0 [-7.625195152111127]
S -> NP BIN_"S->NP,ADVP,VP"_0 [-5.420259037353279]
S -> NP BIN_"S->NP,NT_,,S-ADV,NT_,,VP,PUNC."_0 [-6.584947404554982]
S -> NP BIN_"S->NP,PUNC``,ADJP-PRD"_0 [-7.763421389039857]
S -> NP BIN_"S->NP,ADVP-TMP,VP,PUNC.,PUNC''"_0 [-8.940313234763122]
S -> NP BIN_"S->NP,PRN,VP"_0 [-8.318342332671072]
S -> NP BIN_"S->NP,NT_,,SBAR-ADV,NT_,,VP,PUNC."_0 [-8.52724352726974]
S -> NP BIN_"S->NP,NP-TMP,VP"_0 [-7.971966140932914]
S -> NP BIN_"S->NP,NT_,,SBAR-ADV,NT_,,VP"_0 [-8.804875263868018]
S -> NP BIN_"S->NP,VP,PUNC.,PUNC-RRB-"_0 [-9.152071463852208]
S -> NP BIN_"S->NP,VP,PUNC.,PUNC''"_0 [-4.449791688808578]
S -> NP BIN_"S->NP,PUNC``,NP-PRD"_0 [-8.940313234763122]
S -> NP BIN_"S->NP,PUNC``,VP,PUNC."_0 [-8.940313234763122]
S -> NP BIN_"S->NP,VP,PUNC:"_0 [-7.763421389039857]
S -> NP BIN_"S->NP,VP,PP"_0 [-8.940313234763122]
S -> NP BIN_"S->NP,NT_,,ADVP,NT_,,VP,PUNC."_0 [-6.417949022440221]
S -> NP BIN_"S->NP,NT_,,S-ADV,NT_,,VP"_0 [-8.80487526